## 감독 코드 추출

In [ ]:
import requests
import json

director_name = input()
director_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json?key=d99e474ec53733a372d8413bd2753506'+'&peopleNm='+director_name
res = requests.get(director_list_url)
text = res.text

d = json.loads(text)
print(d)

정이삭
{'faultInfo': {'message': '키의 하루 이용량을 초과하였습니다.', 'errorCode': '320011'}}


In [ ]:
director_list = []
filmo_list = []

for director in d['peopleListResult']['peopleList']:
  if director['repRoleNm'] == '감독':
    filmo_list = director['filmoNames'].split('|')
    director_list.append([director['peopleCd'],director['peopleNm'],director['peopleNmEn'],
                     director['repRoleNm'],filmo_list])
  

print(director_list[0][0])
print(director_list)

KeyError: ignored

In [ ]:
import pandas as pd

director_df = pd.DataFrame(director_list)
director_df.columns = ['peopleCd','peopleNm','peopleNmEn','repRoleNm','filmoNames']
# director_df
print(director_df['filmoNames'].iloc[0][0].__class__)
print(director_df['filmoNames'].iloc[0][:5])
print(director_df['filmoNames'].values.flatten())
# director_df['filmoNames'] = director_df['filmoNames'].astype(str)
# print(director_df['filmoNames'].head())
print(director_df['peopleCd'][0])

## 영화 코드 추출

In [ ]:
movieCd_list = []
for director in director_df['peopleCd']:
  movie_url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json?key=d99e474ec53733a372d8413bd2753506'+'&peopleCd='+director
  res = requests.get(movie_url)
  text = res.text
  d = json.loads(text)
  for movie in d['peopleInfoResult']['peopleInfo']['filmos']:
    if movie['moviePartNm'] == '감독':
      movieCd_list.append([movie['movieCd'], movie['movieNm']])
  print(d)
  print(movieCd_list)

In [ ]:
movieCd_df = pd.DataFrame(movieCd_list)
movieCd_df.columns = ['movieCd', 'movieNm']
movieCd_df

## 평점 추출

In [ ]:
import os
import sys
import urllib.request
from urllib.parse import quote
import json
import re
import requests

In [ ]:
client_id = "5ujvhVMfhcKiunU70W0w"
client_secret = "BwcbJNwH8b"

In [ ]:
film_list = []
for movieNm in movieCd_df['movieNm']:
  encText = urllib.parse.quote(movieNm)
  url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText + "&display=100" # json 결과
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()
  if(rescode==200):
    response_body = json.load(response)
    film_list.append(response_body)
    print(response_body)
  else:
    print("Error Code:" + rescode)

print(film_list)

In [ ]:
print(type(film_list[0]))

In [ ]:
film_df = pd.DataFrame(film_list)
film_df

In [ ]:
film_df.astype({'display': int})

In [ ]:
rating_list = []

for i in range(film_df.shape[0]):
  for j in range(film_df['display'][i]):
    if director_name in film_df['items'][i][j]['director']:
      rating_list.append([film_df['items'][i][j]['title'].replace("<b>","").replace("</b>",""),film_df['items'][i][j]['link'],film_df['items'][i][j]['image'],
                          film_df['items'][i][j]['subtitle'].replace("<b>","").replace("</b>",""),
                          film_df['items'][i][j]['pubDate'],film_df['items'][i][j]['director'],film_df['items'][i][j]['actor'],film_df['items'][i][j]['userRating']])

rating_list

In [ ]:
rating_df = pd.DataFrame(rating_list)
rating_df.columns=['movieNm','link','image','submovieNm','pubDate','director','actor','userRating']
rating_df = rating_df.drop_duplicates(['movieNm'], keep='first', ignore_index=True)
rating_df_sorted = rating_df.sort_values(by=['userRating'], axis=0, ascending=False)

In [ ]:
rating_df

In [ ]:
rating_df_sorted

In [ ]:
final_recommendation = rating_df_sorted.head(5)
final_recommendation
final_list = final_recommendation.values.tolist()
print(final_list)
print(len(final_list))